In [69]:
import os
from datetime import datetime, timedelta
from ftplib import FTP
import paramiko
import stat
import logging
# from django.core.management.base import BaseCommand
from klim4cast_server_settings import sftp_server, sftp_user, sft_port, sftp_password

In [70]:
logging.basicConfig(
    filename='ftp_download.log',
    level=logging.INFO,
    format='%(asctime)s - %(message)s'
)

In [79]:
def download_directory(sftp, remote_dir, local_dir):
    """
    The data on the ftp server is organized in directories in the /data folder.
    The directories are named with the iso dates. This function takes as input the sftp connection, 
    the remote directory (e.g. 'data/2024-06-30') and the local path where the data should be downloaded to ('./../../data/chech_globe_data/<ISO-date>).
    """
    for entry in sftp.listdir_attr(remote_dir):
        remote_path = f"{remote_dir}/{entry.filename}"
        local_path = os.path.join(local_dir, entry.filename)
        if stat.S_ISDIR(entry.st_mode):
            os.makedirs(local_path, exist_ok=True)
            download_directory(sftp, remote_path, local_path)
        else:
            sftp.get(remote_path, local_path)


In [97]:
download_directory(sftp, 'data/2024-06-30', './../../data/chech_globe_data/2024-06-30')

In [71]:
def is_iso_date(filename):
    """
    Checks if a string is a valid ISO 8601 date
    """
    try:
        datetime.fromisoformat(filename)
        return True
    except ValueError:
        return False

In [94]:
def list_dates(sftp):
    """
    List all the dates in the 'data' directory on the sftp server
    """    
    file_paths = []
    dates = []
    sftp_list = sftp.listdir_attr('data')
    for file in sftp_list:
        if stat.S_ISDIR(file.st_mode) and is_iso_date(file.filename):
            dates.append(file.filename)
            
    dates = list_dates(sftp)
    latest_date = sorted(dates)[-1]

    return dates, latest_date

In [95]:
def main():
    """
    Connect to the CheckGlobe FTP server and download the latest data.
    Creation of NetCDF file from downloaded tif files.
    """

    try:
        transport = paramiko.Transport((sftp_server, sft_port))
        transport.connect(username=sftp_user, password=sftp_password)
        sftp = paramiko.SFTPClient.from_transport(transport)
        print("Connection successfully established ...")

        dates, latest_date = list_dates(sftp)

        local_dirs = os.listdir('./../../data/chech_globe_data')
        if local_dirs[-1] < latest_date:
            try:
                remote_dir = f"data/{latest_date}"
                local_dir = f"./../../data/chech_globe_data/{latest_date}"
                download_directory(sftp, remote_dir, local_dir)
            except Exception as e:
                print(e)
                print("Download from CheckGlobe FTP failed ...")
                os.rmdir(local_dir)
            finally:
            # Close the SFTP connection
                sftp.close()
                transport.close()
                print("CheckGlobe FTP Connection closed ...")
    except Exception as e:
        print(e)
        print("Connection to CheckGlobe FTP failed ...")
        
                




In [120]:
local_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# chech_globe_dir = os.path.abspath.join(local_dir, 'data', 'check_globe_data')

NameError: name '__file__' is not defined